## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-29-20-52-32 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/09/29/world/...
1,2025-09-29-20-51-17 +0000,nypost,"Putin orders 135,000 men to join the military ...",https://nypost.com/2025/09/29/world-news/putin...
2,2025-09-29-20-50-55 +0000,nyt,Read Each Point of Trump’s Plan for an Israel-...,https://www.nytimes.com/2025/09/29/world/middl...
3,2025-09-29-20-48-43 +0000,nypost,Michigan church shooter Thomas Sanford’s famil...,https://nypost.com/2025/09/29/us-news/michigan...
4,2025-09-29-20-47-31 +0000,startribune,Twins fire manager Rocco Baldelli a day after ...,https://www.startribune.com/minnesota-twins-fi...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2317/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
13,trump,42
69,new,19
20,church,16
19,michigan,16
50,government,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
252,2025-09-29-01-58-54 +0000,nypost,President Trump shows off new White House blin...,https://nypost.com/2025/09/28/us-news/presiden...,97
125,2025-09-29-15-31-00 +0000,wsj,President Trump said films made outside the U....,https://www.wsj.com/politics/elections/trump-r...,90
17,2025-09-29-20-25-03 +0000,startribune,Trump and Netanyahu say they've agreed on a pl...,https://www.startribune.com/netanyahu-and-trum...,82
86,2025-09-29-17-11-11 +0000,nypost,Kamala Harris reveals plans for hypothetical a...,https://nypost.com/2025/09/29/us-news/kamala-h...,80
185,2025-09-29-11-20-37 +0000,bbc,Trump to push new peace plan in talks with Net...,https://www.bbc.com/news/articles/cvgr092lnyyo...,79


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
252,97,2025-09-29-01-58-54 +0000,nypost,President Trump shows off new White House blin...,https://nypost.com/2025/09/28/us-news/presiden...
85,77,2025-09-29-17-11-23 +0000,nypost,Michigan church shooter Thomas Sanford had pri...,https://nypost.com/2025/09/29/us-news/michigan...
29,54,2025-09-29-19-45-00 +0000,wsj,Democratic congressional leaders head to the W...,https://www.wsj.com/politics/policy/government...
51,54,2025-09-29-18-55-45 +0000,nypost,Frank founder Charlie Javice sobs in court as ...,https://nypost.com/2025/09/29/business/charlie...
143,47,2025-09-29-14-37-00 +0000,wsj,"Eric Adams Ends Re-Election Bid, Reshaping Rac...",https://www.wsj.com/us-news/new-york-city-mayo...
35,44,2025-09-29-19-32-00 +0000,wsj,The clock is ticking down to an Oct. 1 deadlin...,https://www.wsj.com/politics/policy/government...
17,40,2025-09-29-20-25-03 +0000,startribune,Trump and Netanyahu say they've agreed on a pl...,https://www.startribune.com/netanyahu-and-trum...
82,36,2025-09-29-17-26-39 +0000,nypost,Armed man busted after plowing car through pol...,https://nypost.com/2025/09/29/us-news/armed-ma...
176,36,2025-09-29-12-05-00 +0000,wsj,Moldova’s Pro-Europe Party Claims Victory Over...,https://www.wsj.com/world/europe/moldovas-pro-...
209,34,2025-09-29-09-56-00 +0000,wsj,Businesses in the eurozone became less gloomy ...,https://www.wsj.com/economy/eurozone-businesse...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
